In [5]:
!conda install -c pytorch pytorch torchvision torchaudio cpuonly scikit-learn

^C


In [ ]:
!pip install polars

In [ ]:
# References
# https://scikit-learn.org/stable/modules/naive_bayes.html
# https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

In [1]:
import torch
import sklearn

In [17]:
from sklearn.datasets import fetch_20newsgroups
import polars as pl

# Load data
categories = ['alt.atheism', 'soc.religion.christian','comp.graphics', 'sci.med']
data = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

#  Create dataframe
df = pl.DataFrame(data.data)
df = df.insert_column(len(df.columns),pl.Series(data.target))

# Set labels
#labels = list(data.feature_names)
#labels.append("class")
df.columns = ["data","val"]

In [18]:
df.head()

data,val
str,i64
"""From: sd345@ci…",1
"""From: ani@ms.u…",1
"""From: djohnson…",3
"""From: s0612596…",3
"""From: stanly@g…",3


In [27]:
# Data Split
from sklearn.model_selection import train_test_split

X = data.data
Y = data.target
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [29]:
trainlen, testlen = len(X_train), len(X_test)
trainlen, testlen

(1805, 452)

In [32]:
# Preprocessing
# Bag of words
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X)
X_train_counts.shape
count_vect.vocabulary_.get(u'algorithm')

4690

In [35]:
# TF-IDF

from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(2257, 35788)

In [39]:
# Naive Bayes application
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, data.target)
docs_new = ['God is love', 'OpenGL on the GPU is fast']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

# Prediction with TF-IDF and Bag of Words
prediction = clf.predict(X_new_tfidf)
for doc, category in zip(docs_new, prediction):
    print('%r => %s' % (doc, data.target_names[category]))


'God is love' => soc.religion.christian
'OpenGL on the GPU is fast' => comp.graphics
